In [1]:
import json
import openai
import datetime
from langame import LangameClient
c = LangameClient(path_to_config_file="../config.yaml")

In [2]:
from langame.conversation_starters import get_existing_conversation_starters
import logging
logger = logging.getLogger("generation")
memes, index, embeddings_model = get_existing_conversation_starters(
    c._firestore_client, logger=logger, confirmed=True)

Skipping id: 4XlXSjPhH6QSGZDzwIQe, garbage
Skipping id: CYsOWtp1ed2XDeplB5O5, garbage
Skipping id: DlXglThZUiPJcuyLLKHV, garbage
Skipping id: NtXdp7ODn4sR3YR2eEW6, garbage
Skipping id: U0DUXOSpfqYsgurOyL3L, garbage
Skipping id: Uikgs6eO85MRivL0peL6, garbage
Skipping id: XIPyRwJloK6ZdAdTHOeu, garbage
Skipping id: d9FxG5haHnN2ku7spAng, garbage
Skipping id: eyzSmD1AdfKkwOYtfoof, garbage
Skipping id: f1ojlkGYgnOI4MLRGC4T, garbage
Skipping id: fySI8xbjfC5hx7QdHpva, garbage
Skipping id: gLDxkmHFL3OFufeje6E8, garbage


Using 16 omp threads (processes), consider increasing --nb_cores if you have more
Launching the whole pipeline 02/28/2022, 13:21:51
There are 1363 embeddings of dim 768
	Compute estimated construction time of the index 02/28/2022, 13:21:51
		-> Train: 16.7 minutes
		-> Add: 0.0 seconds
		Total: 16.7 minutes
	>>> Finished "Compute estimated construction time of the index" in 0.0000 secs
	Checking that your have enough memory available to create the index 02/28/2022, 13:21:51
5.0MB of memory will be needed to build the index (more might be used if you have more)
	>>> Finished "Checking that your have enough memory available to create the index" in 0.0004 secs
	Selecting most promising index types given data characteristics 02/28/2022, 13:21:51
	>>> Finished "Selecting most promising index types given data characteristics" in 0.0000 secs
	Creating the index 02/28/2022, 13:21:51
		-> Instanciate the index HNSW15 02/28/2022, 13:21:51
		>>> Finished "-> Instanciate the index HNSW15" in 0.000

100%|██████████| 1/1 [00:00<00:00, 592.25it/s]

		>>> Finished "-> Adding the vectors to the index" in 0.0562 secs
	>>> Finished "Creating the index" in 0.0581 secs
	Computing best hyperparameters 02/28/2022, 13:21:51


	>>> Finished "Computing best hyperparameters" in 3.2208 secs
The best hyperparameters are: efSearch=15745
	Compute fast metrics 02/28/2022, 13:21:54
2000
	>>> Finished "Compute fast metrics" in 6.8558 secs
	Saving the index on local disk 02/28/2022, 13:22:01
	>>> Finished "Saving the index on local disk" in 0.0026 secs
Recap:
{'99p_search_speed_ms': 12.475840533152223,
 'avg_search_speed_ms': 3.423994009383023,
 'compression ratio': 0.9576151833909592,
 'index_key': 'HNSW15',
 'index_param': 'efSearch=15745',
 'nb vectors': 1363,
 'reconstruction error %': 0.0,
 'size in bytes': 4372462,
 'vectors dimension': 768}
>>> Finished "Launching the whole pipeline" in 10.1472 secs


In [3]:
out_file_name = f"../data/fine_tune_generation_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in memes:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"{','.join(e['topics'])} ###",
            "completion": f" {e['content']}\n",
        }, outfile)
        outfile.write('\n')

In [4]:
!head -n3 $out_file_name | jq .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "prompt": "ice breaker ###",
  "completion": " When is a time when you know for sure you'll soon have to exchange a nice, meaningful conversation with someone new?\n"
}
{
  "prompt": "space exploration,space travel ###",
  "completion": " Do you think humans are the only intelligent life in the universe?\n"
}
{
  "prompt": "ecology ###",
  "completion": " Have natural disasters gotten worse with the increase in human existence? If so, why?\n"
}


In [5]:
!openai tools fine_tunes.prepare_data -f $out_file_name

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Analyzing...

- Your file contains 1363 prompt-completion pairs
- There are 26 duplicated prompt-completion sets. These are rows: [243, 410, 412, 434, 436, 474, 497, 567, 601, 604, 619, 646, 735, 818, 820, 842, 851, 956, 966, 1024, 1036, 1083, 1149, 1262, 1304, 1316]
- All prompts end with suffix ` ###`
- All completions end with suffix `\n`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 26 duplicate rows [Y/n]: ^C



In [ ]:
# split to train and validation files
with open(out_file_name, "r") as infile:
    lines = infile.readlines()
    lines = lines[:int(len(lines) * 0.8)]
    with open(out_file_name.replace(".jsonl", "_train.jsonl"), "w") as outfile:
        outfile.writelines(lines)
    lines = lines[int(len(lines) * 0.8):]
    with open(out_file_name.replace(".jsonl", "_valid.jsonl"), "w") as outfile:
        outfile.writelines(lines)
        
train = openai.File.create(
  file=open(out_file_name.replace(".jsonl", "_train.jsonl")),
  purpose="fine-tune"
)
validation = openai.File.create(
  file=open(out_file_name.replace(".jsonl", "_valid.jsonl")),
  purpose="fine-tune"
)

ft = openai.FineTune.create(
    training_file=train["id"],
    validation_file=validation["id"],
    model="curie",
)

In [ ]:
results = openai.FineTune.retrieve(
    ft["id"],
)
results